In [1]:
from functions import *
import numpy as np

In [2]:
%load_ext autoreload
%autoreload 2

In [6]:
n10 = [
    (0.1488743389816312108848260, 0.2955242247147528701738930),
    (0.4333953941292471907992659, 0.2692667193099963550912269),
    (0.6794095682990244062343274, 0.2190863625159820439955349),
    (0.8650633666889845107320967, 0.1494513491505805931457763),
    (0.9739065285171717200779640, 0.0666713443086881375935688),
    (-0.1488743389816312108848260, 0.2955242247147528701738930),
    (-0.4333953941292471907992659, 0.2692667193099963550912269),
    (-0.6794095682990244062343274, 0.2190863625159820439955349),
    (-0.8650633666889845107320967, 0.1494513491505805931457763),
    (-0.9739065285171717200779640, 0.0666713443086881375935688),
]

In [149]:
def produto_interno_φ(k, q, xi, h, i, j, pontos_e_pesos_φ):
    # Calcula os produtos internos de φij

    if abs(i-j)>1:
        return 0
    else:
        # Multiplicação de φi e φj para integral definida em [x-1,x]
        φi_φj1 = lambda x, y: k(x) * (1/h) * (1/h) + q(x) * (x-xi[i-1])/h * (x-xi[j-1])/h
        # Multiplicação de φi e φj para integral definida em [x,x+1]
        φi_φj2 = lambda x, y: k(x) * (-1/h) * (-1/h) + q(x) * (xi[i+1]-x)/h * (xi[j+1]-x)/h
        
        # Faz a integral definida em [x-1,x]
        integral1 = integral_dupla(φi_φj1, xi[i-1], xi[i], pontos_e_pesos_φ)

        # Faz a integral definida em [x,x+1]
        integral2 = integral_dupla(φi_φj2, xi[i], xi[i+1], pontos_e_pesos_φ)

        return integral1 + integral2


In [154]:
def φ_xi(i,xi,h):
    φ1 = lambda x: (x-xi[i-1])/h
    φ2 = lambda x: (xi[i+1]-x)/h
    return φ1, φ2


In [145]:
def produto_interno_f_φ(f,xi,i,h,pontos_e_pesos_f_φ):
    # Calcula os produtos internos entre f(x) e φ(x)

    # Multiplica f(x) por φ(x) para integral definida em [x-1,x]
    f_φ1 = lambda x, y: (x - xi[i-1])/h * f(x)

    # Multiplica f(x) por φ(x) para integral definida em [x,x+1]
    f_φ2 = lambda x, y: (xi[i+1] - x)/h * f(x)

    # Faz a integral definida em [x-1,x]
    integral1 = integral_dupla(f_φ1, xi[i-1], xi[i], pontos_e_pesos_f_φ)

    # Faz a integral definida em [x,x+1]
    integral2 = integral_dupla(f_φ2, xi[i], xi[i+1], pontos_e_pesos_f_φ)

    return integral1 + integral2

In [160]:
def monta_sistema_tridigonal(n, pontos_e_pesos_φ):
    A = np.zeros((n, n), dtype=np.float64)
    h = 1/(n+1)
    xi = [i*h for i in range(n+2)]
    for i in range(n):
        for j in range(n):
            A[i][j] = produto_interno_φ(lambda x: 1, lambda x: 0, xi, h, i+1, j+1, pontos_e_pesos_φ)
    return A

monta_sistema_tridigonal(4,n10)

array([[10., 10.,  0.,  0.],
       [10., 10., 10.,  0.],
       [ 0., 10., 10., 10.],
       [ 0.,  0., 10., 10.]])

In [ ]:
def produto_interno_φ(k, q, xi, h, i, j, pontos_e_pesos_φ):
    # Calcula os produtos internos de φij

    if abs(i-j)>1:
        return 0
    else:
        # Multiplicação de φi e φj para integral definida em [x-1,x]
        φi_φj1 = lambda x, y: k(x) * (1/h) * (1/h) + q(x) * (x-xi[i-1])/h * (x-xi[j-1])/h
        # Multiplicação de φi e φj para integral definida em [x,x+1]
        φi_φj2 = lambda x, y: k(x) * (-1/h) * (-1/h) + q(x) * (xi[i+1]-x)/h * (xi[j+1]-x)/h
        
        # Faz a integral definida em [x-1,x]
        integral1 = integral_dupla(φi_φj1, xi[i-1], xi[i], pontos_e_pesos_φ)

        # Faz a integral definida em [x,x+1]
        integral2 = integral_dupla(φi_φj2, xi[i], xi[i+1], pontos_e_pesos_φ)

        return integral1 + integral2


In [3]:
def calcula_produto_interno_phis_diagonal_principal(
    k, q, past_xi, current_xi, next_xi, h, pontos_e_pesos_φ
):

    # Calcula o produto interno de φi e φi
    φi_φi_1 = lambda x, y: k(x, y) + (x - past_xi) ** 2 * q(x, y)
    φi_φi_2 = lambda x, y: k(x, y) + (next_xi - x) ** 2 * q(x, y)
    return (1 / h) ** 2 * (
        integral_dupla(φi_φi_1, past_xi, current_xi, pontos_e_pesos_φ)
        + integral_dupla(φi_φi_2, current_xi, next_xi, pontos_e_pesos_φ)
    )


def calcula_produto_interno_phis_diagonal_secundaria_inferior(
    k, q, past_xi, current_xi, h, pontos_e_pesos_φ
):

    # Calcula o produto interno de φi e φi+1
    φi_φj = lambda x, y: k(x, y) + (current_xi - x) * (x - past_xi) * q(x, y)
    return -((1 / h) ** 2) * integral_dupla(
        φi_φj, past_xi, current_xi, pontos_e_pesos_φ
    )


def calcula_produto_interno_phis_diagonal_secundaria_superior(
    k, q, current_xi, next_xi, h, pontos_e_pesos_φ
):

    # Calcula o produto interno de φi e φi-1
    φj_φi = lambda x, y: k(x, y) + (next_xi - x) * (x - current_xi) * q(x, y)
    return -((1 / h) ** 2) * integral_dupla(
        φj_φi, current_xi, next_xi, pontos_e_pesos_φ
    )


def calcula_produto_interno_f_phi(
    f, past_xi, current_xi, next_xi, h, pontos_e_pesos_φ
):

    # Calcula o produto interno de f(x) e φi
    f_φi1 = lambda x, y: (x - past_xi) * f(x,y)
    f_φi2 = lambda x, y: (next_xi - x) * f(x,y)
    return (
        1
        / h
        * (
            integral_dupla(f_φi1, past_xi, current_xi, pontos_e_pesos_φ)
            + integral_dupla(f_φi2, current_xi, next_xi, pontos_e_pesos_φ)
        )
    )


In [4]:
# Resolve sistema tridiagonais
def sistemaTridiagLU(a, b, c, d, n):
    # Calcula vetores u e l
    u = [b[0]]
    l = []
    for i in range(1, n):
        l.append(a[i] / u[i - 1])
        u.append(b[i] - l[i - 1] * c[i - 1])

    # Calcula solução de L*y = d
    y = [d[0]]
    for i in range(1, n):
        y.append(d[i] - l[i - 1] * y[i - 1])

    # Calcula solução de U*x = y
    x = [0] * n
    x[n - 1] = y[n - 1] / u[n - 1]
    for i in reversed(range(0, n - 1)):
        x[i] = (y[i] - c[i] * x[i + 1]) / u[i]

    return x

In [27]:
def sol_sistema_linear_tridiagonal(f, k, q, n, pontos_e_pesos_φ):
    A = np.zeros((n, n), dtype=np.float64)
    h = 1/(n+1)
    xi = [i*h for i in range(n+2)]
    
    # Vetores do sistema linear (tridigonal)
    b=[] # diagonal principal
    a=[] # diagonal secundária inferior
    c=[] # diagonal secundária superior
    d=[] # vetor de termos independentes
    for i in range(n):
        past_xi = xi[i]
        current_xi=xi[i+1]
        next_xi=xi[i+2]

        b.append(calcula_produto_interno_phis_diagonal_principal(k,q,past_xi,current_xi,next_xi,h,pontos_e_pesos_φ))
        a.append(calcula_produto_interno_phis_diagonal_secundaria_inferior(k,q,past_xi,current_xi,h,pontos_e_pesos_φ))
        c.append(calcula_produto_interno_phis_diagonal_secundaria_superior(k,q,current_xi,next_xi,h,pontos_e_pesos_φ))
        d.append(calcula_produto_interno_f_phi(f, past_xi, current_xi, next_xi, h, pontos_e_pesos_φ))

    # Resolve o sistema linear
    alphas = np.array(sistemaTridiagLU(a, b, c, d, n))

    return alphas

for n in [7,15,31,63,127,255]:
    n=n
    L=1
    f=lambda x,y: np.exp(x)+1
    k=lambda x,y: np.exp(x)
    q=lambda x,y: 0
    alphas = sol_sistema_linear_tridiagonal(f, k, q, n, n10)

    x=0.1
    h = L/(n+1)
    xi = [i*h for i in range(n+2)]
    u_exato = lambda x: (x-1)*(np.exp(-x)-1)
    print(erro(x, u_barra(x,alphas,xi,h),u_exato))

-0.0034377965259725918
-0.0012483368487786473
-0.00020614263558747792
-7.748230204625284e-05
-1.3018124052369129e-05
-4.851056143562182e-06


In [297]:
n=7
L=1
f=lambda x,y: 
k=lambda x,y: 1
q=lambda x,y: 0
alphas = sol_sistema_linear_tridiagonal(f, k, q, n, n10)

x=0.1
h = L/(n+1)
xi = [i*h for i in range(n+2)]
u_exato = lambda x: x**2*(1-x)**2
erro(x, u_barra(x,alphas,xi,h),u_exato)

array([0.00081491, 0.0030651 , 0.00647217, 0.01077573, 0.01573332,
       0.02112044, 0.02673055, 0.03237509, 0.03788344, 0.04310293,
       0.04789888, 0.05215455, 0.05577115, 0.05866788, 0.06078187,
       0.06206822, 0.0625    , 0.06206822, 0.06078187, 0.05866788,
       0.05577115, 0.05215455, 0.04789888, 0.04310293, 0.03788344,
       0.03237509, 0.02673055, 0.02112044, 0.01573332, 0.01077573,
       0.00647217, 0.0030651 , 0.00081491])

In [298]:
u_barra(0.1,alphas,xi,h)

0.008193598017265236

In [9]:
def phi(x,xi,i,h):
    if xi[i-1]<x<=xi[i]:
        return (x-xi[i-1])/h
    elif xi[i]<x<=xi[i+1]:
        return (xi[i+1]-x)/h
    else:
        return 0

def u_barra(x,alphas,xi,h):
    u_barra = 0
    for i in range(len(alphas)):
        u_barra += alphas[i] * phi(x,xi,i+1,h)
    return u_barra

def erro(x, u_barra, u_exato):
    return u_barra-u_exato(x)

In [247]:
for i in range(7):
    print(xi[i+1])

0.125
0.25
0.375
0.5
0.625
0.75
0.875


In [165]:
xi

[0.0, 0.2, 0.4, 0.6000000000000001, 0.8, 1.0]

In [30]:
h = 1 / (n + 1)
xi = [i * h for i in range(n + 2)]
erro = 0
for x in xi:
        u_bar = u_barra(x, alphas, xi, h)
        if abs(u_bar - u_exato(x)) > erro:
            u_bar_saved = u_bar
            erro = abs(u_bar - u_exato(x))
print(
                f"u_barra = {u_bar_saved:.22f} ; u_exato = {u_exato(x):.22f}; erro = {erro:.22f}"
            )

u_barra = 0.1988317873343936614816 ; u_exato = -0.0000000000000000000000; erro = 0.0000000971096521251713
